In [1]:
import os
import numpy as np
import pandas as pd

from src import RESULT_DIR
from src.utils import MCC, DIS

In [6]:
classifier = 'RandomForest' # RandomForest, LightGBM
alignment = 'warp' # geometric, temporal, warp

In [7]:
if classifier == 'LightGBM': 
        cols = ['fold_num', 'trial', 'split_num', 'num_leaves', 
                'min_child_samples', 'threshold', 'num_trees', 
                'tn', 'fp', 'fn', 'tp','time']
        cc = ['fold_num', 'trial', 'num_leaves', 'min_child_samples', 
              'threshold']
        
elif classifier == 'RandomForest': 
        cols = ['num_fold', 'step', 'num_split', 'trees', 
                'max_depth', 'threshold',  
                'tn', 'fp', 'fn', 'tp','time']
        cc = ['num_fold', 'step', 'trees', 'max_depth', 
              'threshold']

In [8]:
res_dir = os.path.join(RESULT_DIR, classifier, alignment, 'classification')
res_df = {}
res_mcc = []
for fold in range(1,10):
    csv_dir = os.path.join(res_dir, 'results_fold{0:02d}.csv'.format(fold))
    dd = pd.read_csv(csv_dir, index_col=0)
    dd = dd[cols]
    dd.reset_index(drop=True, inplace=True)
    #dd['trial'] = np.repeat(list(range(int(dd.shape[0] / 8))),8).astype(int)
    dd = dd.groupby(cc).aggregate({
                    'tn':np.sum,'fp':np.sum,'fn':np.sum,
                    'tp':np.sum,'time':np.average}).reset_index()
    dd['MCC'] = dd.apply(lambda x: MCC(x['tn'],x['fp'],x['fn'],x['tp']), axis=1)
    dd_max = dd.loc[dd['MCC'] == dd['MCC'].max(), :]
    #res_df[fold] = {'num_leaves':dd_max['num_leaves'].values[0], 'min_child_samples':dd_max['min_child_samples'].values[0], 'threshold':dd_max['threshold'].values[0]}
    print('Fold {0}:  {1:>3} iterations - max it. = {6}, max.MCC = {2:.4f}  [{3}  {4}  {5:0.4f}]'.format(fold,dd[cc[1]].max()+1,dd['MCC'].max(),
                                                                                                         dd_max[cc[2]].values[0],
                                                                                                         dd_max[cc[3]].values[0],
                                                                                                         dd_max['threshold'].values[0],
                                                                                                         dd_max[cc[1]].values[0]))
    #res_mcc.append(dd['MCC'].max())


Fold 1:  300 iterations - max it. = 282, max.MCC = 0.7035  [200  60  40.0000]
Fold 2:  300 iterations - max it. = 297, max.MCC = 0.7164  [200  60  39.0000]
Fold 3:  300 iterations - max it. = 109, max.MCC = 0.6913  [200  44  38.0000]
Fold 4:  300 iterations - max it. = 197, max.MCC = 0.7297  [200  55  40.0000]
Fold 5:  300 iterations - max it. = 232, max.MCC = 0.6958  [195  52  42.0000]
Fold 6:  300 iterations - max it. = 40, max.MCC = 0.7280  [200  54  41.0000]
Fold 7:  300 iterations - max it. = 271, max.MCC = 0.7484  [200  65  41.0000]
Fold 8:  300 iterations - max it. = 97, max.MCC = 0.7112  [199  72  40.0000]
Fold 9:  300 iterations - max it. = 150, max.MCC = 0.7140  [200  54  40.0000]
